In [7]:
#Import packages
from polygon.rest import RESTClient
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import numpy as np

#Import API keys from file
polygon_api_key = open("polygon_io_api_key.txt").read()
alphavantage_api_key = open("alphavantage_api_key.txt").read()

#Ticker
ticker = "crm".upper()

# Calls the API to get historical stock data
stock_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={alphavantage_api_key}'
stock_r = requests.get(stock_url)
stock_data = stock_r.json()

earnings_url = f"https://api.polygon.io/benzinga/v1/earnings?ticker={ticker}&limit=100&sort=date.desc&apiKey={polygon_api_key}&date_status=confirmed"
earnings_r = requests.get(earnings_url)
earnings_data = earnings_r.json()

In [8]:
##############
# Stock Data #
##############

# Extract the time series data
time_series = stock_data['Time Series (Daily)']

# Convert the time series data into a DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')

# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Convert the columns to numeric
df = df.apply(pd.to_numeric)

df = df.reset_index().rename(columns={'index': 'tradingDate'})

#Earnings
df_len = []
for i in range(len(earnings_data["results"])):
    df_len.append(i)

benzinga_earnings = pd.DataFrame(earnings_data["results"], index=df_len)
df_column_drop = ['actual_eps','estimated_eps','previous_eps','eps_surprise_percent','revenue_surprise_percent','currency','date_status','benzinga_id','importance','company_name','estimated_revenue','previous_revenue','last_updated','eps_surprise','eps_method','actual_revenue','revenue_surprise','revenue_method',]
benzinga_earnings.drop(df_column_drop, axis='columns', inplace=True)

noon = dt.time(12,0,0)
market_periods = []

benzinga_earnings.loc[:, "time"] = pd.to_datetime(benzinga_earnings["time"], format='%H:%M:%S').dt.time

for time in benzinga_earnings["time"]:
    if time >= noon:
        market_periods.append('after-market')
    else:
        market_periods.append('pre-market')

benzinga_earnings.loc[:,'market_period'] = market_periods
benzinga_earnings.drop('time',axis="columns", inplace=True)
benzinga_earnings = benzinga_earnings.head(30)

# Function to adjust the date based on reportTime
def adjust_date(row):
   if row['market_period'] == 'after-market':
       return df[df["tradingDate"] > row['date']].min(axis=0)["tradingDate"]
   elif row['market_period'] == 'pre-market':
       return df[df["tradingDate"] < row['date']].max(axis=0)["tradingDate"]
   else:
       return row['date']

benzinga_earnings['date'] = pd.to_datetime(benzinga_earnings['date'])

# Apply the function to create a new column with adjusted dates
# Last earnings is used to determine how the stock moved historically around earnings
benzinga_earnings['adjustedDate'] = benzinga_earnings.apply(adjust_date, axis=1)

#############
# merged_df #
#############

#merged_df joins the first set of earnings data
merged_df = pd.merge(benzinga_earnings, df, left_on='date', right_on="tradingDate", how='left')
columns_to_drop = ['tradingDate','5. volume']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df = merged_df.rename(columns={
   '1. open': 'report open',
   '2. high': 'report high',
   '3. low': 'report low',
   '4. close': 'report close'
})

###############
# merged_2_df #
###############

#merged_2_df is the final joined dataset that combines the first set of earnings data with the second set of earnings data
merged_2_df = pd.merge(merged_df, df, left_on='adjustedDate', right_on="tradingDate", how='left')

merged_2_df.drop(columns=columns_to_drop, inplace=True)

merged_2_df = merged_2_df.rename(columns={
   '1. open': 'adjusted open',
   '2. high': 'adjusted high',
   '3. low': 'adjusted low',
   '4. close': 'adjusted close'
})

################
# calculations #
################

# used to calculate the difference between the high and low depending is earnings was pre-market or post-market
# and if the stock went up or down

# Initialize a new column with NaN values to calculate the difference between high and low
merged_2_df['diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   #Post-market
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted low'] - row['report high'],2)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted high'] - row['report low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan
   #Pre-market
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report low'] - row['adjusted high'],2)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report high'] - row['adjusted low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan

# Initialize a new column with NaN values for the percentage calculation
merged_2_df['percent_diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   if row['market_period'] == 'after-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round((row['adjusted low'] / row['report high'])-1,3)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['adjusted high'] / row['report low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan
   elif row['market_period'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report low'] / row['adjusted high']-1,3)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report high'] / row['adjusted low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan

merged_2_df.sort_values(by="percent_diff", ascending=False)

,fiscal_period,fiscal_year,ticker,date,notes,market_period,adjustedDate,report open,report high,report low,report close,adjusted open,adjusted high,adjusted low,adjusted close,diff,percent_diff
20,Q2,2021,CRM,2020-08-25,NaN,after-market,2020-08-26,213.22,218.3500,210.800,216.05,251.96,277.9700,249.4700,272.32,67.17,0.319
10,Q4,2023,CRM,2023-03-01,NaN,after-market,2023-03-02,162.99,167.9800,162.980,167.35,193.12,193.9100,185.2000,186.59,30.93,0.190
13,Q1,2023,CRM,2022-05-31,NaN,after-market,2022-06-01,166.96,167.2800,159.350,160.24,178.01,184.4200,174.3700,176.07,25.07,0.157
3,Q3,2025,CRM,2024-12-03,NaN,after-market,2024-12-04,327.40,332.7999,323.650,331.43,366.81,369.0000,356.6500,367.87,45.35,0.140
27,Q3,2019,CRM,2018-11-27,NaN,after-market,2018-11-28,125.15,127.6100,123.720,127.54,138.48,140.8900,131.9700,140.64,17.17,0.139
25,Q1,2020,CRM,2019-06-04,NaN,after-market,2019-06-05,146.05,150.9200,142.510,150.81,157.15,158.5000,154.5500,158.44,15.99,0.112
7,Q3,2024,CRM,2023-11-29,NaN,after-market,2023-11-30,228.61,231.5000,227.260,230.35,249.05,252.5000,243.6500,251.90,25.24,0.111
8,Q2,2024,CRM,2023-08-30,NaN,after-market,2023-08-31,211.77,215.5900,209.430,215.04,228.00,228.7900,221.2000,221.46,19.36,0.092
24,Q2,2020,CRM,2019-08-22,NaN,after-market,2019-08-23,149.49,149.9000,145.470,148.24,157.70,158.4200,150.6500,151.57,12.95,0.089
17,Q1,2022,CRM,2021-05-27,NaN,after-market,2021-05-28,229.52,229.5200,225.390,225.83,239.22,243.5000,237.8300,238.10,18.11,0.080
